In [ ]:
import pandas as pd
import json
from datetime import datetime

# Load JSON files
with open("/home/baskar/Desktop/Research/StockPricePred/datasets/NASDAQ_100-20250521T124016Z-1-001/NASDAQ_100/ABNB_10Y_DAILY.json", "r") as f:
    daily_data = json.load(f)

with open("/home/baskar/Desktop/Research/StockPricePred/datasets/FMP-20250521T123934Z-1-001/FMP/AAPL_10Y_QUARTERLY.json", "r") as f:
    quarterly_data = json.load(f)

# Convert daily data to DataFrame
daily_df = pd.DataFrame(daily_data)
daily_df['date'] = pd.to_datetime(daily_df['date'])

# Convert quarterly data to DataFrame
quarterly_df = pd.DataFrame(quarterly_data)
quarterly_df['date'] = pd.to_datetime(quarterly_df['date'])

# Define helper to assign quarter based on date
def get_quarter(d):
    month = d.month
    if month in [1, 2, 3]:
        return "Q1"
    elif month in [4, 5, 6]:
        return "Q2"
    elif month in [7, 8, 9]:
        return "Q3"
    else:
        return "Q4"

# Add year and quarter columns to daily_df
daily_df['year'] = daily_df['date'].dt.year
daily_df['quarter'] = daily_df['date'].apply(get_quarter)

# Add year and quarter columns to quarterly_df for merging
quarterly_df['year'] = quarterly_df['calendarYear'].astype(int)
quarterly_df['quarter'] = quarterly_df['period']

# Merge on year and quarter
merged_df = pd.merge(daily_df, quarterly_df, how='left', on=['year', 'quarter'])
# Drop unnecessary columns (optional)
merged_df.drop(columns=['calendarYear', 'period'], inplace=True)

# View result
merged_df


,date_x,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,...,priceToSalesRatio,priceEarningsRatio,priceToFreeCashFlowsRatio,priceToOperatingCashFlowsRatio,priceCashFlowRatio,priceEarningsToGrowthRatio,priceSalesRatio,dividendYield,enterpriseValueMultiple,priceFairValue
0,2020-12-10 00:00:00+00:00,144.71,165.0000,141.2500,146.00,70447537,144.71,165.0000,141.2500,146.00,...,29.602612,37.781697,101.917294,93.080764,93.080764,2.728678,29.602612,0.001833,109.099622,29.312199
1,2020-12-11 00:00:00+00:00,139.25,151.5000,135.1000,146.55,26980830,139.25,151.5000,135.1000,146.55,...,29.602612,37.781697,101.917294,93.080764,93.080764,2.728678,29.602612,0.001833,109.099622,29.312199
2,2020-12-14 00:00:00+00:00,130.00,135.3000,125.1601,135.00,16966089,130.00,135.3000,125.1601,135.00,...,29.602612,37.781697,101.917294,93.080764,93.080764,2.728678,29.602612,0.001833,109.099622,29.312199
3,2020-12-15 00:00:00+00:00,124.80,127.6000,121.5000,126.69,10914432,124.80,127.6000,121.5000,126.69,...,29.602612,37.781697,101.917294,93.080764,93.080764,2.728678,29.602612,0.001833,109.099622,29.312199
4,2020-12-16 00:00:00+00:00,137.99,142.0000,124.9100,125.83,20409551,137.99,142.0000,124.9100,125.83,...,29.602612,37.781697,101.917294,93.080764,93.080764,2.728678,29.602612,0.001833,109.099622,29.312199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1035,2025-01-27 00:00:00+00:00,128.96,129.0600,125.6400,125.94,4806967,128.96,129.0600,125.6400,125.94,...,31.011567,26.525859,142.794511,128.770263,128.770263,0.178681,31.011567,0.001000,85.407689,57.741961
1036,2025-01-28 00:00:00+00:00,131.05,131.4200,126.8600,128.67,5929900,131.05,131.4200,126.8600,128.67,...,31.011567,26.525859,142.794511,128.770263,128.770263,0.178681,31.011567,0.001000,85.407689,57.741961
1037,2025-01-29 00:00:00+00:00,132.58,134.0500,130.8900,131.42,3627926,132.58,134.0500,130.8900,131.42,...,31.011567,26.525859,142.794511,128.770263,128.770263,0.178681,31.011567,0.001000,85.407689,57.741961
1038,2025-01-30 00:00:00+00:00,131.79,134.3714,130.8500,133.56,2440541,131.79,134.3714,130.8500,133.56,...,31.011567,26.525859,142.794511,128.770263,128.770263,0.178681,31.011567,0.001000,85.407689,57.741961


In [8]:
import json

def load_fundamentals_as_timesteps(fundamentals_json_path, use='date'):
    with open(fundamentals_json_path, 'r') as f:
        fundamentals = json.load(f)
    
    results = []
    for entry in fundamentals:
        timestamp = entry.get(use)
        # Extract only the ratio fields (exclude metadata)
        ratio_fields = {
            k: v for k, v in entry.items()
            if k not in {"symbol", "date", "calendarYear", "period"}
        }

        ratio_vector = list(ratio_fields.values())
        results.append([timestamp, ratio_vector])

    return results


samples = load_fundamentals_as_timesteps("/home/baskar/Desktop/Research/StockPricePred/datasets/FMP-20250521T123934Z-1-001/FMP/AAPL_10Y_QUARTERLY.json")
print(f"Total samples: {len(samples[0][1])}")
print("Example sample:", samples[0])

Total samples: 54
Example sample: ['2025-03-29', [0.8208700223419635, 0.7775072455748386, 0.19479702014926922, 46.999444205581014, 11.174245425017824, 58.17368963059884, 96.47746177612295, -38.303772145524114, 0.47050619238876246, 0.31029058610094484, 0.3073648003859101, 0.25986010759341016, 0.15455475946775846, 0.07481138654663032, 0.3709802982214504, 0.1585164629115728, 0.8454452405322416, 0.9905708202372503, 0.31029058610094484, 0.2964257788324231, 1.4699383196598599, 0.5404851336662951, 0.5951315901128608, 0, 0.24394516529851507, 4.958874782921133, 1.9149162616972568, 0.9328603628570373, 8.054235125219334, 2.034281935318713, 0.2878910011985521, 1.5974302394771485, 1.392616100138708, 3.234476108640729, 0.1516545601291364, 0.25117713063266184, 0.8717852371409486, 0.24394516529851507, 1.2207951070336391, 7.799413871703028, 3.5073949333723826, 0.1516545601291364, 48.913265282352235, 48.913265282352235, 34.262214031187405, 32.962171789749796, 156.4681034337436, 136.40658265698062, 136.4

In [7]:
import os
import json
import pandas as pd
from pathlib import Path

def load_json_files_from_folder(folder_path):
    dataframes = []
    for file in Path(folder_path).glob('*.json'):
        try:
            with open(file, 'r') as f:
                data = json.load(f)
                if isinstance(data, list):
                    df = pd.DataFrame(data)
                else:
                    df = pd.DataFrame([data])
                df['__source_file__'] = file.name
                dataframes.append(df)
        except Exception as e:
            print(f"Error reading {file}: {e}")
    return dataframes

def perform_eda(df):
    eda = {
        'Shape': df.shape,
        'Columns': df.columns.tolist(),
        'Data Types': pd.Series(df.dtypes).reset_index().rename(columns={'index': 'Column', 0: 'Type'}),
        'Missing Values': df.isnull().sum().reset_index().rename(columns={'index': 'Column', 0: 'Missing Count'}),
        'Unique Counts': df.nunique().reset_index().rename(columns={'index': 'Column', 0: 'Unique Values'}),
        'Summary Stats': df.describe(include='all').transpose()
    }
    return eda

def display_eda(folder_name, eda):
    print(f"\n=== EDA Summary for Folder: {folder_name} ===\n")

    print(f"Shape: {eda['Shape']}")
    print("\nColumns:")
    print(eda['Columns'])

    print("\n>> Data Types:")
    display(eda['Data Types'])

    print("\n>> Missing Values:")
    display(eda['Missing Values'])

    print("\n>> Unique Value Counts:")
    display(eda['Unique Counts'])

    print("\n>> Summary Statistics:")
    display(eda['Summary Stats'])

def run_eda_on_folders(folder1, folder2):
    for folder in [folder1, folder2]:
        if not os.path.exists(folder):
            print(f"Folder not found: {folder}")
            continue
        dfs = load_json_files_from_folder(folder)
        if dfs:
            combined_df = pd.concat(dfs, ignore_index=True)
            eda = perform_eda(combined_df)
            display_eda(folder, eda)
        else:
            print(f"No readable JSON files in {folder}")

# === Example usage ===
if __name__ == "__main__":
    folder1_path = "/home/baskar/Desktop/Research/StockPricePred/datasets/FMP-20250521T123934Z-1-001/FMP"
    folder2_path = "/home/baskar/Desktop/Research/StockPricePred/datasets/NASDAQ_100-20250521T124016Z-1-001/NASDAQ_100"
    run_eda_on_folders(folder1_path, folder2_path)



=== EDA Summary for Folder: /home/baskar/Desktop/Research/StockPricePred/datasets/FMP-20250521T123934Z-1-001/FMP ===

Shape: (10893, 59)

Columns:
['symbol', 'date', 'calendarYear', 'period', 'currentRatio', 'quickRatio', 'cashRatio', 'daysOfSalesOutstanding', 'daysOfInventoryOutstanding', 'operatingCycle', 'daysOfPayablesOutstanding', 'cashConversionCycle', 'grossProfitMargin', 'operatingProfitMargin', 'pretaxProfitMargin', 'netProfitMargin', 'effectiveTaxRate', 'returnOnAssets', 'returnOnEquity', 'returnOnCapitalEmployed', 'netIncomePerEBT', 'ebtPerEbit', 'ebitPerRevenue', 'debtRatio', 'debtEquityRatio', 'longTermDebtToCapitalization', 'totalDebtToCapitalization', 'interestCoverage', 'cashFlowToDebtRatio', 'companyEquityMultiplier', 'receivablesTurnover', 'payablesTurnover', 'inventoryTurnover', 'fixedAssetTurnover', 'assetTurnover', 'operatingCashFlowPerShare', 'freeCashFlowPerShare', 'cashPerShare', 'payoutRatio', 'operatingCashFlowSalesRatio', 'freeCashFlowOperatingCashFlowRatio'

,Column,Type
0,symbol,object
1,date,object
2,calendarYear,object
3,period,object
4,currentRatio,float64
5,quickRatio,float64
6,cashRatio,float64
7,daysOfSalesOutstanding,float64
8,daysOfInventoryOutstanding,float64
9,operatingCycle,float64



>> Missing Values:


,Column,Missing Count
0,symbol,0
1,date,0
2,calendarYear,0
3,period,0
4,currentRatio,0
5,quickRatio,0
6,cashRatio,0
7,daysOfSalesOutstanding,615
8,daysOfInventoryOutstanding,78
9,operatingCycle,615



>> Unique Value Counts:


...

,Column,Missing Count
0,date,0
1,close,0
2,high,0
3,low,0
4,open,0
5,volume,0
6,adjClose,0
7,adjHigh,0
8,adjLow,0
9,adjOpen,0



>> Unique Value Counts:


,Column,Unique Values
0,date,13889
1,close,179089
2,high,193002
3,low,191932
4,open,175627
5,volume,359783
6,adjClose,395954
7,adjHigh,397220
8,adjLow,396360
9,adjOpen,389231



>> Summary Statistics:


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
date,613437,13889,2025-01-17T00:00:00.000Z,96,NaN,NaN,NaN,NaN,NaN,NaN,NaN
close,613437.0,NaN,NaN,NaN,1350.928252,5347.011428,0.62,51.38,161.44,739.17,134657.28
high,613437.0,NaN,NaN,NaN,1366.971984,5402.132968,0.62,52.25,164.0,749.0,134884.8
low,613437.0,NaN,NaN,NaN,1334.052841,5287.437571,0.5938,50.5,158.999921,729.0,133764.48
open,613437.0,NaN,NaN,NaN,1350.801673,5346.574885,0.62,51.375,161.48,739.06,134496.0
volume,613437.0,NaN,NaN,NaN,5297937.849491,20761003.624807,0.0,478745.0,1596735.0,4317700.0,9232320000.0
adjClose,613437.0,NaN,NaN,NaN,68.141743,187.949652,0.006458,5.427208,18.778898,57.453541,5289.474426
adjHigh,613437.0,NaN,NaN,NaN,69.027742,190.152543,0.006458,5.523434,19.065,58.176297,5326.298782
adjLow,613437.0,NaN,NaN,NaN,67.221941,185.678783,0.006185,5.328526,18.477476,56.67,5249.217122
adjOpen,613437.0,NaN,NaN,NaN,68.135584,187.934149,0.006458,5.427208,18.779884,57.410424,5289.135123


In [8]:
import os
import json
import pandas as pd
from pathlib import Path

def load_json_files_from_folder(folder_path):
    dataframes = []
    for file in Path(folder_path).glob('*.json'):
        try:
            with open(file, 'r') as f:
                data = json.load(f)
                if isinstance(data, list):
                    df = pd.DataFrame(data)
                else:
                    df = pd.DataFrame([data])
                dataframes.append(df)
        except Exception as e:
            print(f"Error reading {file}: {e}")
    return dataframes

def generate_label(df, price_col='close', date_col='date'):
    if date_col not in df.columns or price_col not in df.columns:
        print("Missing required columns for labeling.")
        return df

    df[date_col] = pd.to_datetime(df[date_col])
    df.sort_values(by=date_col, inplace=True)

    # Calculate percent change over 65 days (quarterly)
    df['quarterly_change'] = df[price_col].pct_change(periods=65) * 100

    def classify(change):
        if pd.isna(change):
            return None
        elif change > 5:
            return 'buy'
        elif change < -5:
            return 'sell'
        else:
            return 'hold'

    df['label'] = df['quarterly_change'].apply(classify)
    return df

def analyze(df):
    original_missing = df.isnull().sum()
    missing_features = (original_missing > 0).sum()

    substituted = 0
    for col in df.columns:
        if df[col].isnull().any():
            if df[col].dtype in ['float64', 'int64']:
                df[col].fillna(df[col].mean(), inplace=True)
                substituted += 1
            elif df[col].dtype == 'object':
                df[col].fillna(df[col].mode().iloc[0] if not df[col].mode().empty else "unknown", inplace=True)
                substituted += 1

    class_counts = df['label'].value_counts(normalize=True) * 100 if 'label' in df.columns else pd.Series(dtype=float)

    return df.shape, missing_features, substituted, class_counts.round(2)

def summarize_folder(folder_path, df):
    df = generate_label(df)

    shape, missing_count, substituted_count, class_dist = analyze(df)

    print(f"\n=== Summary for Folder: {folder_path} ===")
    
    # Display as pandas objects
    shape_df = pd.DataFrame({'Dataset Shape': [shape]})
    missing_df = pd.DataFrame({'Features with Missing Values': [missing_count]})
    substituted_df = pd.DataFrame({'Features Substituted (Filled)': [substituted_count]})
    
    print("\nDataset Shape:")
    display(shape_df)

    print("\nFeatures with Missing Values:")
    display(missing_df)

    print("\nFeatures Substituted (Filled):")
    display(substituted_df)

    if not class_dist.empty:
        print("\nClass Balance based on derived label (quarterly price change):")
        display(class_dist.reset_index().rename(columns={'index': 'Label', 'label': 'Percentage (%)'}))
    else:
        print("No valid label could be generated.")

def run_eda_on_folders(folder1, folder2):
    for folder in [folder1, folder2]:
        if not os.path.exists(folder):
            print(f"Folder not found: {folder}")
            continue

        dfs = load_json_files_from_folder(folder)
        if dfs:
            combined_df = pd.concat(dfs, ignore_index=True)
            summarize_folder(folder, combined_df)
        else:
            print(f"No JSON files found in {folder}")

# === Example usage ===
if __name__ == "__main__":
    folder1_path = "/home/baskar/Desktop/Research/StockPricePred/datasets/FMP-20250521T123934Z-1-001/FMP"
    folder2_path = "/home/baskar/Desktop/Research/StockPricePred/datasets/NASDAQ_100-20250521T124016Z-1-001/NASDAQ_100"
    run_eda_on_folders(folder1_path, folder2_path)


Missing required columns for labeling.

=== Summary for Folder: /home/baskar/Desktop/Research/StockPricePred/datasets/FMP-20250521T123934Z-1-001/FMP ===

Dataset Shape:


/tmp/ipykernel_284462/810099777.py:53: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mean(), inplace=True)


,Dataset Shape
0,"(10893, 58)"



Features with Missing Values:


,Features with Missing Values
0,23


...

,Features with Missing Values
0,2



Features Substituted (Filled):


,Features Substituted (Filled)
0,2



Class Balance based on derived label (quarterly price change):


,Percentage (%),proportion
0,buy,48.29
1,sell,48.21
2,hold,3.51
